In [23]:
import numpy as np
import pandas as pd
from scipy import stats
from stldecompose import decompose, forecast
from statsmodels.tsa.seasonal import seasonal_decompose
import ciso8601


from . import thermometr


SystemError: Parent module '' not loaded, cannot perform relative import

In [ ]:
client = Elasticsearch()

query = {
    "query": {
        'bool': {
            'filter': [{
                'range': {
                    '@timestamp': {
                        'gte': '2018-07-11T03:00:00.000Z', 
                        'lte': '2018-08-17T03:00:00.000Z'}
                    }
                }]
            }
        }, 
        'aggs': {
            'my_aggregation': {
                'date_histogram': {
                    'interval': '1H', 
                    'field': '@timestamp'
                }
        }
    }
}

response = client.search(
    index="ts-movi*",
    size=0,
    body=query)

print("Query time: ", response['took'])

In [ ]:
jsondata= []
for element in response['aggregations']['my_aggregation']['buckets']:
    jsondata.append([element['key'], element['doc_count']])
        
dates, count = zip(*jsondata)
s = pd.Series(count, index=dates)

s.head(20)

In [ ]:
dates = []
count = []
for element in response['aggregations']['my_aggregation']['buckets']:
    dates.append(pd.to_datetime(element['key_as_string']))
    count.append(element['doc_count'])
    
ts = pd.Series(count, index=dates)
ts.head(20)

In [ ]:
stl = decompose(ts, period=336)
stl.seasonal.head(336)